<a href="https://colab.research.google.com/github/pedrogzz18/ASL-Model/blob/models_/Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.metrics import accuracy_score

In [3]:
import string
letters = list(string.ascii_uppercase)
letters

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z']

In [4]:
numbers = [str(num) for num in range(0, 10)]
numbers

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [5]:
class_names = letters + numbers
class_names

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [66]:
df = pd.read_csv('coords.csv')

<ipython-input-66-9f8ffc621bcb>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('coords.csv')


In [67]:
df.dropna(axis = 0, inplace = True)

In [68]:
df['class'] = pd.Series(df['class'], dtype=str)

In [69]:
df_alpha = df[df['class'].map(lambda c : c.isalpha())]
df_numeric = df[df['class'].map(lambda c : c.isnumeric())]

In [70]:
X_alpha = df_alpha.drop('class', axis = 1)
y_alpha = df_alpha['class']
X_numeric = df_numeric.drop('class', axis = 1)
y_numeric = df_numeric['class']

In [71]:
alpha_encoder = LabelEncoder()
y_alpha = alpha_encoder.fit_transform(y_alpha)
print(y_alpha)

numeric_encoder = LabelEncoder()
y_numeric = numeric_encoder.fit_transform(y_numeric)
print(y_numeric)

[ 0  0  0 ... 23 23 23]
[0 0 0 ... 8 8 8]


In [72]:
X_alpha_train, X_alpha_test, y_alpha_train, y_alpha_test = train_test_split(X_alpha, y_alpha,
                                                                            test_size = 0.3,
                                                                            random_state = 42,
                                                                            shuffle=True,
                                                                            stratify = y_alpha)
X_numeric_train, X_numeric_test, y_numeric_train, y_numeric_test = train_test_split(X_numeric, y_numeric,
                                                                            test_size = 0.3,
                                                                            random_state = 42,
                                                                            shuffle=True,
                                                                            stratify = y_numeric)

In [128]:
alpha_class_models = {
    'lr_alpha': make_pipeline(StandardScaler(), LogisticRegression(max_iter = 300)),
    'rc_alpha': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gbc_alpha': make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

numeric_class_models = {
    'lr_numeric': make_pipeline(StandardScaler(), LogisticRegression(max_iter = 300)),
    'rc_numeric': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gbc_numeric': make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

alpha_fit_models = {}
numeric_fit_models = {}

for algo, model in alpha_class_models.items():
    alpha_fit_models[algo] = model.fit(X_alpha_train, y_alpha_train)

for algo, model in numeric_class_models.items():
    numeric_fit_models[algo] = model.fit(X_numeric_train, y_numeric_train)

In [95]:
for algo, model in alpha_fit_models.items():
    preds = model.predict(X_alpha_test)
    acc = accuracy_score(y_alpha_test, preds)
    print(algo, acc)

for algo, model in numeric_fit_models.items():
    preds = model.predict(X_numeric_test)
    acc = accuracy_score(y_numeric_test, preds)
    print(algo, acc)

lr_alpha 0.9861111111111112
rc_alpha 0.9788194444444445
gbc_alpha 0.9625
lr_numeric 0.9902777777777778
rc_numeric 0.9884259259259259
gbc_numeric 0.9851851851851852


## Neural Network

In [ ]:
!pip install scikeras tensorflow

In [83]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Dropout
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

In [121]:
def create_alpha_asl_nn():
  model = Sequential([
      Dense(input_shape=(84,), units=84, activation='relu'),
      Dense(units=32, activation='relu'),
      Dense(units=len(letters), activation='softmax')
  ])
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
  model.summary()
  return model

def create_numeric_asl_nn():
  model = Sequential([
      Dense(input_shape=(84,), units=84, activation='relu'),
      Dense(units=32, activation='relu'),
      Dense(units=len(numbers), activation='softmax')
  ])
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=["accuracy"])
  model.summary()
  return model

In [122]:
alpha_nn_classifier = KerasClassifier(build_fn = create_alpha_asl_nn, epochs=200, batch_size=200, shuffle=True, verbose=1, validation_split=0.2 )
alpha_nn = make_pipeline(StandardScaler(), alpha_nn_classifier)

numeric_nn_classifier = KerasClassifier(build_fn = create_numeric_asl_nn, epochs=100, batch_size=100, shuffle=True, verbose=1, validation_split=0.2 )
numeric_nn = make_pipeline(StandardScaler(), numeric_nn_classifier)

In [123]:
alpha_nn_classifier.fit(X_alpha_train, y_alpha_train)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_86 (Dense)                     │ (None, 84)                  │           7,140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_87 (Dense)                     │ (None, 32)                  │           2,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_88 (Dense)                     │ (None, 26)                  │             858 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,718 (41.87 KB)

 Trainable params: 10,718 (41.87 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.0740 - loss: 3.2354 - val_accuracy: 0.1257 - val_loss: 3.1447
Epoch 2/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1594 - loss: 3.0883 - val_accuracy: 0.1763 - val_loss: 2.9144
Epoch 3/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2123 - loss: 2.8181 - val_accuracy: 0.3039 - val_loss: 2.5703
Epoch 4/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3328 - loss: 2.4491 - val_accuracy: 0.3698 - val_loss: 2.2122
Epoch 5/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4311 - loss: 2.0921 - val_accuracy: 0.4423 - val_loss: 1.9172
Epoch 6/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5150 - loss: 1.8088 - val_accuracy: 0.5063 - val_loss: 1.6838
Epoch 7/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5586 - loss: 1.6039 - val_accuracy: 0.5465 - val_loss: 1.5060
Epoch 8/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6048 - loss: 1.4306 - val_accuracy: 0.6016 - v

KerasClassifier(
	model=None
	build_fn=<function create_alpha_asl_nn at 0x7ff003ca1bd0>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=200
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=200
	class_weight=None
)

In [124]:
numeric_nn_classifier.fit(X_numeric_train, y_numeric_train)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_89 (Dense)                     │ (None, 84)                  │           7,140 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_90 (Dense)                     │ (None, 32)                  │           2,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_91 (Dense)                     │ (None, 10)                  │             330 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,190 (39.80 KB)

 Trainable params: 10,190 (39.80 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.1572 - loss: 2.2779 - val_accuracy: 0.2321 - val_loss: 2.1948
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2763 - loss: 2.1558 - val_accuracy: 0.3869 - val_loss: 2.0577
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3908 - loss: 1.9971 - val_accuracy: 0.3522 - val_loss: 1.8489
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4148 - loss: 1.7618 - val_accuracy: 0.5615 - val_loss: 1.5883
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5730 - loss: 1.4981 - val_accuracy: 0.6052 - val_loss: 1.3539
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6505 - loss: 1.2732 - val_accuracy: 0.7123 - val_loss: 1.1592
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7278 - loss: 1.0780 - val_accuracy: 0.7808 - val_loss: 1.0011
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7699 - loss: 0.9497 - val_accuracy: 0.7897 - v

KerasClassifier(
	model=None
	build_fn=<function create_numeric_asl_nn at 0x7ff003ca1990>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=100
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=100
	class_weight=None
)

In [125]:
preds = alpha_nn_classifier.predict(X_alpha_test)
print(preds, y_alpha_test)
acc = accuracy_score(y_alpha_test, preds)
print('NN', acc)

29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
[10 20 15 ...  8 19 22] [10 20 15 ...  8 19 22]
NN 0.9723958333333333


In [126]:
preds = numeric_nn_classifier.predict(X_numeric_test)
print(preds, y_numeric_test)
acc = accuracy_score(y_numeric_test, preds)
print('NN', acc)

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
[4 7 4 ... 4 4 8] [4 7 4 ... 4 4 8]
NN 0.9842592592592593


In [129]:
import pickle

# save
with open('alpha_nn_ASL.pkl','wb') as f:
    pickle.dump(alpha_nn_classifier,f)

with open('numeric_nn_ASL.pkl','wb') as f:
    pickle.dump(numeric_nn_classifier,f)

for algo, model in alpha_fit_models.items():
    with open(f'{algo}_ASL.pkl','wb') as f:
      pickle.dump(model,f)

for algo, model in numeric_fit_models.items():
    with open(f'{algo}_ASL.pkl','wb') as f:
      pickle.dump(model,f)

In [131]:
with open('lr_alpha_ASL.pkl', 'rb') as f:
  ASL_letter_classifier = pickle.load(f)
  pred = ASL_letter_classifier.predict([X_alpha_test.iloc[0]])
  print(pred)
  print(y_alpha_test[0])

[10]
10


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
